In [43]:
import json
import pandas as pd

In [44]:
with open('C:/Users/ab356917/Downloads/topgear/Critic/train.json','r',encoding='utf-8', errors='ignore') as file:
    d= json.loads("[%s]"%file.read().strip().replace('\n',','))
    file.close()

In [45]:
for i in range(len(d[:400])):
    with open('./Train/'+str(i)+'.json','w',encoding='utf-8') as f:
        f.write(json.dumps(d[i]).replace("Review","text"))
        f.close()

In [46]:
from watson_developer_cloud import DiscoveryV1

In [47]:
username = ""
password = ""
environment_id = ""
collection_id = ""
discovery = DiscoveryV1(username= username, password= password, version="2017-11-07")

In [162]:
def train_response():
    data= []
    discovery_response = discovery.query(environment_id,collection_id,count = 1000,highlight = "True")
    for result in discovery_response["results"]:
        data_attributes = []
        total_sentiment_score = 0
        length_of_review = len(result["text"])
        data_attributes.append(result["Stars"])
#         for entities in result["enriched_text"]["entities"]:
#             if entities["type"] == "Person":
#                 total_sentiment_score += entities["sentiment"]["score"]
#         total_emotion = result["enriched_text"]["emotion"]["document"]["emotion"]["disgust"]+result["enriched_text"]["emotion"]["document"]["emotion"]["joy"]+result["enriched_text"]["emotion"]["document"]["emotion"]["anger"]+result["enriched_text"]["emotion"]["document"]["emotion"]["fear"]+result["enriched_text"]["emotion"]["document"]["emotion"]["sadness"]
#         data_attributes.append(total_sentiment_score)
#         data_attributes.append(result["enriched_text"]["sentiment"]["document"]["score"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["disgust"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["joy"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["anger"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["fear"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["sadness"])
        data_attributes.append(length_of_review)
#         data_attributes.append(total_emotion)
        #list_of_attributes.append(jsons["Stars"]) 
        data_attributes.append(result["Critic"])
        data.append(data_attributes)  
        
    df  = pd.DataFrame(data =data, columns = ['Stars','disgust','joy','anger','fear','sadness','length_of_review','Critic'])
    print(df)
    df.to_csv("data.csv")

In [49]:
train_response()

     Stars  total_sentiment_score  document_sentiment   disgust       joy  \
0      4.0               3.589214           -0.124926  0.132653  0.538229   
1      2.5               3.179636           -0.241398  0.107063  0.547763   
2      2.5              -0.794239           -0.205072  0.168174  0.575469   
3      3.5               4.613047            0.181308  0.112047  0.569917   
4      3.0               0.971755           -0.229301  0.117921  0.604462   
5      3.0               2.138783            0.139368  0.091713  0.679511   
6      3.0               3.828249            0.133586  0.143281  0.602006   
7      3.5               0.391139           -0.064093  0.136320  0.554119   
8      3.0               2.319557           -0.119857  0.149689  0.608066   
9      3.5               1.650654            0.392289  0.142483  0.575086   
10     2.0               5.753196           -0.213067  0.111498  0.604817   
11     1.5              -4.367178           -0.358412  0.155413  0.520110   

In [50]:
test_dict = d[400:]

In [51]:
Test_data = []
for i in test_dict:
    Test_data.append({"Review":i['Review'],"Stars":i['Stars']})

In [52]:
for i in range(len(Test_data)):
    with open('./Test/'+str(i)+'.json','w',encoding='utf-8') as f:
        f.write(json.dumps(Test_data[i]).replace("Review","text"))
        f.close()

In [163]:
def test_response():
    data= []
    discovery_response = discovery.query(environment_id,"21eb80d7-e4ff-4c7e-87ac-2d347cd0a023",count = 1000,highlight = "True")
    for result in discovery_response["results"]:
        data_attributes = []
        total_sentiment_score = 0
        length_of_review = len(result["text"])
        data_attributes.append(result["Stars"])
#         for entities in result["enriched_text"]["entities"]:
#             if entities["type"] == "Person":
#                 total_sentiment_score += entities["sentiment"]["score"]
#         total_emotion = result["enriched_text"]["emotion"]["document"]["emotion"]["disgust"]+result["enriched_text"]["emotion"]["document"]["emotion"]["joy"]+result["enriched_text"]["emotion"]["document"]["emotion"]["anger"]+result["enriched_text"]["emotion"]["document"]["emotion"]["fear"]+result["enriched_text"]["emotion"]["document"]["emotion"]["sadness"]
#         data_attributes.append(total_sentiment_score)
#         data_attributes.append(result["enriched_text"]["sentiment"]["document"]["score"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["disgust"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["joy"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["anger"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["fear"])
        data_attributes.append(result["enriched_text"]["emotion"]["document"]["emotion"]["sadness"])
        data_attributes.append(length_of_review)
#         data_attributes.append(total_emotion)
        data.append(data_attributes)  
  
    df  = pd.DataFrame(data =data, columns = ['Stars','disgust','joy','anger','fear','sadness','length_of_review'])
    print(df)
    df.to_csv("test_output.csv")

In [55]:
test_response()

WatsonApiException: Error: Could not find listed collection, Code: 404

# Logistic Regression

In [134]:
#Load Data
iris = pd.read_csv('data.csv')
iris.drop('Unnamed: 0',axis=1,inplace=True)
iris.head(4)

Stars  total_sentiment_score  document_sentiment   disgust       joy  \
0    4.0               3.589214           -0.124926  0.132653  0.538229   
1    2.5               3.179636           -0.241398  0.107063  0.547763   
2    2.5              -0.794239           -0.205072  0.168174  0.575469   
3    3.5               4.613047            0.181308  0.112047  0.569917   

      anger      fear   sadness  length_of_review  total_emotion Critic  
0  0.123678  0.092914  0.550971             11105       1.438445  Delta  
1  0.452082  0.528650  0.524070              5105       2.159628  Alpha  
2  0.471383  0.114856  0.557299              4931       1.887181  Alpha  
3  0.131483  0.143408  0.550864              6663       1.507719  Alpha

In [147]:
#Data setup

Species = ['Alpha', 'Beta', 'Gamma','Delta']
#Number of examples
m = iris.shape[0]
#Features
n = 10
#Number of classes
k = 4

X = np.ones((m,n + 1))
y = np.array((m,1))
X[:,1] = iris['Stars'].values
X[:,2] = iris['total_sentiment_score'].values
X[:,3] = iris['document_sentiment'].values
X[:,4] = iris['disgust'].values
X[:,5] = iris['joy'].values
X[:,6] = iris['anger'].values
X[:,7] = iris['fear'].values
X[:,8] = iris['sadness'].values
X[:,9] = iris['length_of_review'].values
X[:,10] = iris['total_emotion'].values

#Labels
y = iris['Critic'].values

#Mean normalization
for j in range(n):
    X[:, j] = (X[:, j] - X[:,j].mean())

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
 

In [148]:
#Logistic Regression

def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

#Regularized cost function
def regCostFunction(theta, X, y, _lambda = 0.1):
    m = len(y)
    h = sigmoid(X.dot(theta))
    tmp = np.copy(theta)
    tmp[0] = 0 
    reg = (_lambda/(2*m)) * np.sum(tmp**2)

    return (1 / m) * (-y.T.dot(np.log(h)) - (1 - y).T.dot(np.log(1 - h))) + reg

#Regularized gradient function
def regGradient(theta, X, y, _lambda = 0.1):
    m, n = X.shape
    theta = theta.reshape((n, 1))
    y = y.reshape((m, 1))
    h = sigmoid(X.dot(theta))
    tmp = np.copy(theta)
    tmp[0] = 0
    reg = _lambda*tmp /m

    return ((1 / m) * X.T.dot(h - y)) + reg

#Optimal theta 
def logisticRegression(X, y, theta):
    result = op.minimize(fun = regCostFunction, x0 = theta, args = (X, y),
                         method = 'TNC', jac = regGradient)
    
    return result.x

In [149]:
#Training

all_theta = np.zeros((k, n + 1))

#One vs all
i = 0
for flower in Species:
    #set the labels in 0 and 1
    tmp_y = np.array(y == flower, dtype = int)
    optTheta = logisticRegression(X, tmp_y, np.zeros((n + 1,1)))
    all_theta[i] = optTheta
    i += 1

In [150]:
#Predictions
P = sigmoid(X.dot(all_theta.T)) #probability for each flower
p = [Species[np.argmax(P[i, :])] for i in range(X.shape[0])]

print("Test Accuracy ", accuracy_score(y, p) * 100 , '%')

Test Accuracy  51.75 %


In [137]:
testing = pd.read_csv('test_output.csv')
testing.drop('Unnamed: 0',axis=1,inplace=True)
testing.head(4)

Stars  total_sentiment_score  document_sentiment   disgust       joy  \
0    2.5              -2.694809           -0.131002  0.118451  0.521800   
1    2.5               1.368488            0.288187  0.094463  0.653318   
2    2.5               0.266893           -0.270354  0.151479  0.560662   
3    3.0               0.567510            0.050417  0.109802  0.573978   

      anger      fear   sadness  length_of_review  total_emotion  
0  0.152872  0.485186  0.507315              6312       1.785624  
1  0.108569  0.127101  0.486469              4056       1.469920  
2  0.149433  0.110077  0.512085              5087       1.483736  
3  0.135179  0.123406  0.577651              9021       1.520016

In [152]:
#Number of examples
o = testing.shape[0]
#Features
n = 10
#Number of classes
k = 4

test = np.ones((o,n + 1))
test[:,1] = testing['Stars'].values
test[:,2] = testing['total_sentiment_score'].values
test[:,3] = testing['document_sentiment'].values
test[:,4] = testing['disgust'].values
test[:,5] = testing['joy'].values
test[:,6] = testing['anger'].values
test[:,7] = testing['fear'].values
test[:,8] = testing['sadness'].values
test[:,9] = testing['length_of_review'].values
test[:,10] = testing['total_emotion'].values


#Mean normalization
for j in range(n):
    test[:, j] = (test[:, j] - test[:,j].mean())

In [153]:
Q = sigmoid(test.dot(all_theta.T)) #probability for each flower
q = [Species[np.argmax(Q[i, :])] for i in range(test.shape[0])]

In [78]:
er=[]
for i in test_dict:
    er.append(i['Critic'])

In [155]:
print("Test Accuracy ", accuracy_score(er, q) * 100 , '%')

Test Accuracy  27.0 %


# KNN

In [135]:
data = iris

In [136]:
data.drop(['total_sentiment_score','document_sentiment','total_emotion','length_of_review'],axis=1,inplace=True)
data.head(3)

Stars   disgust       joy     anger      fear   sadness Critic
0    4.0  0.132653  0.538229  0.123678  0.092914  0.550971  Delta
1    2.5  0.107063  0.547763  0.452082  0.528650  0.524070  Alpha
2    2.5  0.168174  0.575469  0.471383  0.114856  0.557299  Alpha

In [8]:
# Importing libraries
import pandas as pd
import numpy as np
import math
import operator

In [71]:
# Defining a function which calculates euclidean distance between two data points
def euclideanDistance(data1, data2, length):
    distance = 0
    for x in range(length):
        distance += np.square(data1[x] - data2[x])
    return np.sqrt(distance)

# Defining our KNN model
def knn(trainingSet, testInstance, k):
 
    distances = {}
    sort = {}
    length = len(testInstance)
    
    #### Start of STEP 3
    # Calculating euclidean distance between each row of training data and test data
    for x in range(len(trainingSet)):
        #### Start of STEP 3.1
        dist =euclideanDistance(testInstance, trainingSet.iloc[x], length)
        print(dist)
        distances[x] = dist
        #### End of STEP 3.1
 
    #### Start of STEP 3.2
    # Sorting them on the basis of distance
    sorted_d = sorted(distances.items(), key=operator.itemgetter(1))
    #### End of STEP 3.2
 
    neighbors = []
    
    #### Start of STEP 3.3
    # Extracting top k neighbors
    for x in range(k):
        neighbors.append(sorted_d[x][0])
    #### End of STEP 3.3
    classVotes = {}
    
    #### Start of STEP 3.4
    # Calculating the most freq class in the neighbors
    for x in range(len(neighbors)):
        response = trainingSet.iloc[neighbors[x]][-1]
 
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    #### End of STEP 3.4

    #### Start of STEP 3.5
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return(sortedVotes[0][0], neighbors)
    #### End of STEP 3.5

In [138]:
test = testing

In [139]:
test.drop(['total_sentiment_score','document_sentiment','total_emotion','length_of_review'],axis=1,inplace=True)
test.head(3)

Stars   disgust       joy     anger      fear   sadness
0    2.5  0.118451  0.521800  0.152872  0.485186  0.507315
1    2.5  0.094463  0.653318  0.108569  0.127101  0.486469
2    2.5  0.151479  0.560662  0.149433  0.110077  0.512085

In [140]:
data.head(3)

Stars   disgust       joy     anger      fear   sadness Critic
0    4.0  0.132653  0.538229  0.123678  0.092914  0.550971  Delta
1    2.5  0.107063  0.547763  0.452082  0.528650  0.524070  Alpha
2    2.5  0.168174  0.575469  0.471383  0.114856  0.557299  Alpha

In [141]:
check = []
for i in range(len(test)):
    check.append(list(test.iloc[i, :]))

In [158]:
out=[]
for i in range(len(check)):
    out.append(knn(data, check[i], k=5))

1.5514854388
0.304138516689
0.496431973756
1.05901957626
0.627960605682
0.728395344064
0.637071112221
1.05883916716
0.599479147489
1.08332235772
0.577042301839
1.06552726351
0.616770508629
1.13586481327
0.63940074802
0.706673132403
0.618336601234
0.592985944693
0.696229863795
1.55853225134
1.11582541186
0.510871229155
0.606300235853
1.50372440856
0.644112402025
1.16719848419
1.00521537616
0.637697025167
0.634779705745
0.612300245803
0.392867761137
0.689173161337
0.746199580434
0.639073144983
0.403846788095
1.58107166294
0.363781095955
0.386011811844
0.357430985694
1.62509835295
0.652745334143
0.538701084448
1.0221325428
0.693688928591
0.613099932633
1.14867434048
0.63623854925
0.338667303519
1.55823455968
1.58233616941
0.376608510776
1.00758948183
0.741576539538
0.618638378353
0.504334701816
0.654436969086
0.637571941671
1.07671438385
0.602767157725
2.10744122549
0.746317760335
0.657167973303
0.719332092528
0.709208614527
1.07056528777
0.594086933823
0.596740559054
0.707093869833
0.509

0.583333196486
1.0747317824
1.07083588655
1.28817912666
0.613038268319
0.667142470332
0.121174141326
1.07143507959
0.568353269401
0.709001383571
1.00135429916
1.00692699982
0.506926257123
0.505912482102
0.0907884822155
0.624107562481
0.601732956463
0.516279252743
0.505499093304
1.50294107078
0.0613371696282
0.319027957482
0.505669219495
0.633123891595
0.587068909495
0.31778907246
0.067220402446
0.648791150781
1.00645361092
1.10196050844
0.0802210002119
0.693917343973
0.500869419755
0.729281824268
0.509565832975
0.751364059963
1.03863531273
0.605555072299
0.711251596415
0.502738817941
1.50063895679
0.090644948359
0.721412060472
1.05685933429
0.717227865221
1.0656177317
0.856296167991
0.0784257970186
0.620575479733
0.506977826289
0.787586344985
0.627719093304
0.604005789769
1.50170053552
0.626598313854
1.00332920774
0.503827523465
0.64807342693
2.0009908539
1.00125993852
0.67825149531
0.514786026724
1.50279340802
0.708628452081
0.504685485639
1.53529183928
1.05860295811
1.20683439696
1.0

0.504582892762
0.583781499715
0.469058046911
0.114262757301
0.415760098428
0.0604630328052
0.0618477548259
0.502188395479
0.263061575383
2.57371766292
0.40270126095
0.0632073330714
1.07198161067
0.395855857713
1.50170284828
1.14025334604
0.0811102311364
1.2239527507
0.390999343517
0.359826818917
0.527356584474
0.515703858879
1.06320262846
1.06665424284
3.00181437368
0.326697530127
0.510409760643
2.04432442287
0.520071521641
1.57266646774
0.49857794882
1.09745462639
0.502154255409
1.09849463567
1.58040358266
1.55040609493
0.506683352972
0.512759447447
0.685451150054
0.503836782554
0.0599574971459
1.58843860093
0.69918116359
0.640703702142
0.510033149968
0.847370958431
0.636046850123
1.07847630033
0.64279467618
0.509103590773
0.43794584322
1.55463358394
1.53466206158
1.08877367288
0.647022650172
0.386004352589
0.503056054541
1.10429323147
0.52090265226
1.00151968619
1.00380313976
1.07401193146
0.510860666549
0.498459661766
0.515508129267
0.375758534543
0.774917957952
0.560272260776
0.571

0.562955512271
0.728857483271
1.53800486232
1.25538182946
0.65362353638
1.61272609873
2.05754242921
0.592587782827
1.50848085212
1.06396653238
1.05740867969
1.05888534025
1.05157155249
1.55188088226
1.16570084539
1.16447098247
0.5909100535
1.09511389511
1.0639331474
1.07039500752
0.5829510953
0.350487122621
0.508296885654
1.05859545346
1.6085435281
0.747884920434
0.605633276919
1.05068881477
1.05426100627
0.637696353985
0.752344744892
1.06468977011
1.07174104497
0.333772414663
1.57323436498
0.631961735074
2.01329595394
1.13912178136
1.05704103411
0.321255416024
1.05669761764
0.622780223481
1.05846672978
0.594078136292
0.534907419296
0.07135134977
0.130302835975
1.07990731406
0.588022215345
0.83751329211
0.735241166103
1.05207363167
0.6466635849
2.02835042301
0.135303512678
0.426350081115
1.56253083084
0.357414632114
0.552440389937
1.07114881179
0.643226015141
0.70757527231
0.66124502252
1.07357967905
0.623378279537
1.09811666031
0.55234675232
1.08238693445
0.634438292604
1.15205327168


0.503101296814
0.613051508813
1.05274637982
1.55664118477
0.622857077983
0.610224095042
2.52974083983
0.511332105246
1.62979230846
0.510218080534
1.06085438573
3.52394245047
1.00161649885
2.54373125697
2.53699159606
1.04334118181
1.00227856937
0.689196531355
1.08366327174
0.633346128807
2.00236146055
2.06781539987
2.00120086035
0.0854550628693
3.04852512783
1.00249122709
1.06128960193
2.00057572012
1.61885575871
1.50908298222
2.00063885475
1.50201456314
0.667209934566
1.0511455991
1.54168736048
2.05362454128
0.453026954411
2.16487394356
1.52909726686
1.59166588197
1.00190321003
0.398126779407
1.53657281221
1.58318568864
0.102773014469
0.108625372174
0.51011300349
0.50325166149
1.01001612611
0.642636193119
0.646749652171
0.515553523437
0.503306562743
0.503343155173
1.00191681955
1.0613975805
0.505615896884
1.5547048709
0.6119053211
1.05400697353
1.00445389272
0.656166669625
2.00236086241
2.0677426061
1.00335540266
0.668407603755
1.50299151202
0.718343443012
1.50309473076
1.6212338047
0.

...

2.0809262249
2.06849084864
1.13793681546
2.55878939438
2.00244901316
1.13492931212
0.555494069404
2.07174772084
1.59869950287
2.04644898292
2.5569175424
1.56584892796
0.581028594772
2.58527361542
1.56907803186
2.55763525765
1.58398405883
1.5705826606
1.13364591634
3.04778159673
2.5279772282
2.53036598046
1.5889774417
2.07422699743
1.57880608458
2.03012168184
3.05265934832
2.576113283
1.5541141155
0.798966478152
0.523483017902
1.11532404624
3.02890965714
2.55812722742
0.755197980867
0.72570588327
0.72644010936
3.06045035436
2.53525291746
0.623814909452
2.07028537099
2.588646472
2.56556541302
2.57086710949
1.13154358202
1.61128127617
1.07189169889
2.55576591649
3.09812644856
2.03341170768
2.07579597108
2.55611120313
2.55892569435
2.08305590065
2.06504354621
0.749738361594
0.755012365831
1.59660297155
3.00125644838
2.07994996385
0.688341619005
2.53066239728
2.56497961128
1.58064852376
2.55777501453
2.08061437685
2.55769381293
2.05806523114
2.04980199697
1.55747055585
1.56627916512
2.57405

0.516756816963
1.23094894056
0.0329088037157
2.00031573722
2.00166298533
0.632753966079
0.768259896916
0.504912367999
1.00365296098
1.0016814219
1.24392659712
0.106904205638
1.00238143969
0.382305909631
1.50079294445
1.07497642875
1.06297231801
1.06609243633
1.09018824258
0.428302412028
0.347366619119
1.06901251793
2.04247459919
1.00826452546
1.00544984017
1.06633924209
1.00571134152
0.721693516133
0.314061700088
1.13520079022
0.649885966536
1.56582347373
1.50070590925
1.11062895051
1.00271368085
1.00235626604
1.00617193435
1.09947701911
0.526946218359
1.50111397302
1.13334862777
0.0196414746391
1.0030902497
1.00124003524
0.504664752908
1.05354282926
1.50192434993
0.58849104653
1.00311833951
1.63202740628
0.523082923004
1.60631500423
0.506801034067
0.700463140061
0.556193049492
2.0001787421
1.56394475592
1.53510726136
0.514735821878
1.27426698571
0.691496476623
1.07608238911
2.00100880405
1.64798403091
0.72235053828
1.10892602294
1.63478082621
0.0938077356299
2.0502775224
1.60351277376

2.55626173658
0.33046903523
0.826641374347
1.58011084087
2.58087878514
0.724254081219
0.541250244491
0.69740547449
0.716848278087
1.59224593663
1.58256273154
1.12433344928
1.54607009301
0.643371337013
1.60328238155
1.13513609001
2.58480552818
1.11815560388
1.13084189594
1.11965334906
0.633391589394
1.59780837755
1.04362701439
2.14598138152
1.0133238157
1.58606818496
2.03213522831
0.709099402009
1.11910332033
2.58439786758
2.58308586505
1.12200367675
2.09142011876
1.54885211865
1.56655574941
1.6364121735
1.6018801058
1.59584920039
2.06852336808
1.55116260663
1.00512050818
1.63949943106
1.59902836123
0.622654590971
1.54798785909
0.547660162951
0.76732449141
1.58605855286
0.838254363585
1.62758479532
1.54939995307
1.67451365238
1.12126846437
2.52360207593
0.608189530805
1.58740185807
1.61928525014
2.07983209155
0.818202013019
1.68732775596
0.697697480253
1.66391748017
0.596581296024
2.08204380509
2.52364351119
0.561139688481
0.292924404784
2.06274336704
2.07183697633
2.05162391681
1.14984

0.519847624098
0.727673513423
0.672095563189
1.00071760129
0.541507000571
2.00267496337
0.363515773223
0.171027974308
4.02749962322
2.60598250168
2.56242110473
3.53439818342
3.03872005009
3.05737618781
3.03803020034
3.53203656252
3.07203639042
3.53111295311
2.11151179263
1.56514092204
3.04082454789
1.62308115283
3.04137601062
3.00488904709
2.06288656924
3.04389700195
3.05101856648
4.02868434155
1.60476016564
2.09886312593
3.03349143141
4.04867272523
2.05836999569
1.56946295296
1.63809801608
2.05217486635
3.0404965148
3.03774818541
2.54486379826
3.05295819839
2.09584751356
3.03626762717
2.54812883719
4.04076122999
2.54551290456
2.55264615229
2.58998855216
1.17819408958
3.04437587427
2.50485343724
3.58165486117
2.1752283653
3.04169496665
3.5035092341
2.05532308129
2.54273444908
4.08502460928
4.0429667015
2.54644863411
3.54747268726
3.06842530937
3.0305192529
3.06821812231
3.04435132239
3.04122708866
3.53695756047
3.03180111598
0.662256344143
3.05934320627
3.04387922591
2.08438471722
3.06

1.16463359355
0.580190957369
1.57678537383
1.05006735132
0.427435251628
1.10799907282
0.120129656189
0.584328120952
0.0701776720403
0.698861703607
1.15716346417
1.17083627916
3.04048204564
0.162736870678
0.668989188999
2.00153511176
0.0662425722704
1.50208709314
0.0354480615972
1.19134364565
0.708351323236
1.19107122661
1.64351297033
1.61272565724
0.69394639038
0.671802886867
0.805572346414
0.72769979511
0.482792125053
1.64362799383
0.632530610282
0.51894759116
0.679438814431
0.941927065643
0.77797412044
1.16543723235
0.430711524222
0.709975650537
0.607895559927
1.61721510439
1.5981636529
1.17843147965
0.785251160637
0.12991763916
0.683348606431
1.18681934803
0.670832620932
1.11574742502
1.12456767953
1.16162281865
0.690985261976
0.640965998358
0.695300528321
0.60888343275
0.887119678115
0.731736644253
0.672043715562
0.0889707731449
2.04723954866
0.801637527343
0.492618390588
0.504218862131
1.1363403175
0.564569706178
1.50047434589
0.58894245059
1.57905510341
1.12621518964
1.1671344650

In [159]:
knn_res = [i[0] for i in out]

In [160]:
val=0
for i in range(200):
    if knn_res[i]==er[i]:
        val+=1

In [161]:
val/200

0.29